## Douglas Decicino de Andrade

### Nº USP: 10883512

### Projeto 3 Redes Neurais

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten
from keras.applications import MobileNetV2
from keras.preprocessing import image
import numpy as np
import keras

# Questão 1 

#### Vamos criar o modelo indicado no trabalho.

Primeiramente vamos utilizar a mobilenet já treinada no conjunto imagenet, pegando apenas as camadas convolucionais dela.

In [19]:
mobilenetv2_conv = MobileNetV2(weights = 'imagenet', 
                               include_top = False, 
                               input_shape = (128,128,3))

Agora, como pedido, vamos congelar os pesos respectivos as camadas convolucionais, para que eles não sejam atualizados quando formos utilizar o modelo final.

In [20]:
mobilenetv2_conv.trainable = False

In [21]:
mobilenetv2_conv.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

Vamos então adicionar uma camada Flatten para transformar o tensor retornado das camadas convolucionais em um vetor inteiro, assim como adicionar a camada densa com 7 neurônios (porque são 7 classes).

In [22]:
out_new_model = mobilenetv2_conv.output
out_new_model = Flatten()(out_new_model)
out_new_model = Dense(7, activation='softmax')(out_new_model)

Temos então nosso novo modelo de Rede Neural

In [23]:
new_model = Model(inputs = mobilenetv2_conv.input,
                 outputs = out_new_model)

In [24]:
new_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

# Questão 2

#### Vamos agora utilizar a função ImageDataGenerator do Keras para utilizar nossas imagens guardadas, de uma maneira que, a partir delas, temos um gerador de imagens que será utilizado para treinar e testar a base de acordo com nossa preferência, assim como foi feito no exemplo de aula.

In [25]:
train_folder = 'dataset/train'
test_folder = 'dataset/test'

Primeiro vamos pré configurar a função para definir como vamos querer que as imagens de treino sejam geradas. Como pedido no trabalho, vamos utilizar algumas distorções e adicionar também o pré processamento que a Mobilenet exige.

In [30]:
train_gen = image.ImageDataGenerator(
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=10,
    height_shift_range=0.1,
    width_shift_range=0.1,
    preprocessing_function = keras.applications.mobilenet_v2.preprocess_input)

Agora, vamos fazer o mesmo para as imagens do conjunto de teste, apenas aplicando o pre-processamento da Mobilenet.

In [31]:
test_gen = image.ImageDataGenerator(
    preprocessing_function = keras.applications.mobilenet_v2.preprocess_input)

Agora que já definimos como queremos que nossas bases sejam geradas, vamos definir em qual diretório está cada parte dos nossos dados, já utilizando separadamente as funções criadas anteriormente para o conjunto de treino e o conjunto de teste. (Definimos aqui também a quanidade de batchs que vamos utilizar, o tamanho do input, e outras coisas).

In [32]:
#Conjunto de treino
train_dir = train_gen.flow_from_directory(
    train_folder,
    target_size = (128,128),
    batch_size = 32,
    class_mode = 'categorical',
    subset='training')

Found 2284 images belonging to 7 classes.


In [33]:
#Conjunto de teste
test_dir = test_gen.flow_from_directory(
    test_folder,
    target_size = (128,128),
    batch_size = 32,
    class_mode = 'categorical',
    subset='training')

Found 1400 images belonging to 7 classes.


Vamos compilar o modelo.

In [34]:
new_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

Agora sim, vamos treinar o modelo (ao mesmo tempo que validar). Os 'steps_per_epoch' foram definidos dividindo o número total de imagens na base de treino (2284) pelo número de batchs definidos do trabalho, assim, teremos que cada epoch será respectiva ao número original da base (aproximadamente devido a arrondamentos). Assim como fazer a mesma coisa para os 'validation_steps, respectivos aos dados de validação.

In [35]:
new_model.fit_generator(
    train_dir,
    steps_per_epoch=71,
    validation_data=test_dir,
    validation_steps=44,
    epochs = 10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
71/71 [==============================] - 67s 946ms/step - loss: 1.3088 - accuracy: 0.7411 - val_loss: 0.5716 - val_accuracy: 0.8593
Epoch 2/10
71/71 [==============================] - 45s 630ms/step - loss: 0.4988 - accuracy: 0.8797 - val_loss: 0.4393 - val_accuracy: 0.9014
Epoch 3/10
71/71 [==============================] - 40s 559ms/step - loss: 0.3496 - accuracy: 0.9152 - val_loss: 0.9695 - val_accuracy: 0.8257
Epoch 4/10
71/71 [==============================] - 41s 577ms/step - loss: 0.4371 - accuracy: 0.9045 - val_loss: 0.6612 - val_accuracy: 0.8886
Epoch 5/10
71/71 [==============================] - 42s 587ms/step - loss: 0.3020 - accuracy: 0.9285 - val_loss: 0.6987 - val_accuracy: 0.8793
Epoch 6/10
71/71 [==============================] - 40s 563ms/step - loss: 0.4401 - accuracy: 0.9139 - val_loss: 0.6136 - val_accuracy: 0.8914
Epoch 7/10
71/71 [==============================] - 38s 540ms/step

Podemos perceber que para as primeiras epochs, a acurácias tanto no conjunto de treino quanto no conjunto de teste começam baixas, após isso, aumentam consideravelmente ao longo das epochs. Temos que o valor da acurácia respectivo a base de treino e teste não seguem um padrão, pois tem vezes que a acurácia na de treino da maior e as vezes menor que a de teste. Porém, valores mais altos que 91% de acerto acontecem apenas no conjunto de treino, o que já é de se esperar, dado que, neste caso, nosso modelo está sendo avaliado no mesmo conjunto em que foi treinado.